In [1]:
import ee
import numpy as np
import pandas as pd
import pickle

In [2]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWgQ-UwJidbbCMjTbUyPmpKE82RFmNze_xjvAxfbxw2Iz2UUmaCmZ4Q



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [141]:
# sampling paramaters
collectionLandsat = "LANDSAT/LC08/C02/T1_L2" # landsat image collection
cloudCoverL0X = 90 # maximum cloud cover of sampled Landsat images
maxSample = 10000 # maximum samples to extract from each Landsat image
outputScale = 500 # scale of output comparisons , increasing this will eventually exceed memory
subimageSize = 10000 # dimension of subimage we will randomly extract from Landsat image to avoid memory limits
minfracAreaLC = 0.1 # minimum cloud free valid  Landsat subimage fraction
minfracAreaMCD15 = 0.1 # minimum cloud free valid MODIS area in Landsat subimage fraction's unmasked area
minfracAreaLCtoMCD = 0.1 # valid aggregated landsat fraction
minfracValidLC = 0.9 # minimum facation of valid landsat pixels in each aggregated modis pixels to estimate mean values
deltaRow = 1  # skip this many WRS rows when sampling Landsat scenes
deltaPath = 1 # skip this many WRS paths when sampling Landsat scenes
deltaDay = 5 # search this many contiguous days when looking for a cloud free Landsat scene
maxPixels = 300 # max landsat pixels allowed to be aggregated in an output pixel, should be about 20% larger than the ratio of (outputScale/30) (cannot exceed 1024)

#output files
unsatInputFileName = '/home/richard_fernandes/unsatLUTinputLC0X'
unsatOutputFileName = '/home/richard_fernandes/unsatLUToutputLC0X'
satInputFileName = '/home/richard_fernandes/satLUTinputLC0X'
satOutputFileName = '/home/richard_fernandes/satLUToutputLC0X'

# initialize output arrays
unsatLUT = [];
unsatLUTLCinput = [];
unsatLUTinput = [];
unsatLUToutput = [];
satLUT = [];
satLUTLCinput = [];
satLUTinput = [];
satLUToutput = [];
Lai_ann_LX = []
LaiStdDev_ann_LX = []
Fpar_ann_LX = []
FparStdDev_ann_LX = []
codeMOD09_ann_LX = []
codeLC07_ann_LX = []
Year_ann_LX = []
Month_ann_LX = []
Daystart_ann_LX =  []
Dayend_ann_LX =  []
Path_ann_LX = []
Row_ann_LX = []
Year_ann_LXsat = []
Month_ann_LXsat = []
Daystart_ann_LXsat =  []
Dayend_ann_LXsat =  []
Path_ann_LXsat = []
Row_ann_LXsat = []

In [142]:
#function to compute distance from images of pixel coordinates to a line defined by a list to two points
def getDist(image,pts) :
  
    # cast inputs
    image = ee.Image(image)
    pts = ee.List(pts)
    
    # identify coord layers
    x0 = image.select('x')
    y0 = image.select('y')
    
    # identify end points
    x1 = ee.Image.constant(ee.Number(ee.List(pts.get(0)).get(0)))
    x2 = ee.Image.constant(ee.Number(ee.List(pts.get(1)).get(0)))
    y1 = ee.Image.constant(ee.Number(ee.List(pts.get(0)).get(1)))
    y2 = ee.Image.constant(ee.Number(ee.List(pts.get(1)).get(1)))
    denom = ((x2.subtract(x1)).pow(ee.Image.constant(2)).add((y2.subtract(y1)).pow(ee.Image.constant(2)))).pow(ee.Image.constant(0.5))
    
    return ((((x2.subtract(x1)).multiply(y1.subtract(y0))).subtract((x1.subtract(x0)).multiply(y2.subtract(y1)))).abs()).divide(denom)
    



In [143]:
#  Estimate view zenith angle in a Landsat image based on lateral distance from satellite track
#  Assume flat earth and linear track within image
def getVZALC(imageLC) :

    # get point farthest from centroid p1
    points = ee.List(((imageLC.geometry().coordinates()).get(0)))
    dists0 = points.map( lambda point: ee.Geometry.Point(point).distance(imageLC.geometry().centroid()).multiply(-1))
    p1 = ee.List(points.sort(dists0).get(0))
    #print(p1)

    # get point farthest from p1:  p2
    dists1 = points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p1)).multiply(-1))
    p2 = ee.List(points.sort(dists1).get(0))
    #print(p2)

    # get point farthest from p1 and p2: p3
    dists2 =  points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p2)).multiply(-1))
    dists2 =  ee.List.sequence(0,dists1.length().subtract(1),1).map(lambda index: ee.Number(dists1.get(index)).add(ee.Number(dists2.get(index))) )
    p3 = ee.List(points.sort(dists2).get(0))
    #p.ipynb_checkpoints/int(p3)

    # get point farthest from p3: p4
    dists3 =  points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p3)).multiply(-1))
    p4 = ee.List(points.sort(dists3).get(0))
    #print(p4)

    # separate easting and northing coords of points
    ptsE = [p1.get(0) , p2.get(0), p3.get(0), p4.get(0)]
    ptsN = [p1.get(1) , p2.get(1), p3.get(1), p4.get(1)]
    #print(ptsE)
    #print(ptsN)

    # make list of corner points
    pts = [p1,p2,p3,p4]
    #print(pts)

    # separate East and West side corner points
    ptWest = ee.Geometry.Point(ee.List(pts).sort(ptsE).get(0)).transform(imageLC.projection()).coordinates()
    ptNorth = ee.Geometry.Point(ee.List(pts).sort(ptsN).get(3)).transform(imageLC.projection()).coordinates()
    ptSouth = ee.Geometry.Point(ee.List(pts).sort(ptsN).get(0)).transform(imageLC.projection()).coordinates()
    ptEast = ee.Geometry.Point(ee.List(pts).sort(ptsE).get(3)).transform(imageLC.projection()).coordinates()

    ptsWest = ee.List([ptWest, ptNorth])
    ptsEast = ee.List([ptEast ,ptSouth])
    #print(ptsEast)
    #print(ptsWest )
    
    # compute distance between each pixel and line defined by East side corner points 
    imageLCCoords = ee.Image.pixelCoordinates(imageLC.projection()).clip(imageLC.geometry())
    distEast = getDist(imageLCCoords ,ptsEast)
    distWest = getDist(imageLCCoords ,ptsWest)

    # compute VZA assuming +/-7.5degree scan angle 
    return (distEast.divide(distEast.add(distWest)).subtract(ee.Image.constant(0.5)).multiply(ee.Image.constant(7.5)))

In [144]:
# deprecated functions but I leave it here as it is useful

# clip the top and bottom rows of Landsat image  by specified degrees
def clipImageRows(image,clipDegrees):
    image = ee.Image(image);
    clipDegrees = ee.Number(clipDegrees);

    
    
    # get the bounding box of the image and parse co-ords
    imageBBOX = ee.List(image.geometry().bounds().coordinates().get(0));
    long1 = ee.Number(ee.List(imageBBOX.get(0)).get(0))
    long2 = ee.Number(ee.List(imageBBOX.get(1)).get(0))
    long3 = ee.Number(ee.List(imageBBOX.get(2)).get(0))
    long4 = ee.Number(ee.List(imageBBOX.get(3)).get(0))
    long5 = ee.Number(ee.List(imageBBOX.get(4)).get(0))
    lat1 = ee.Number(ee.List(imageBBOX.get(0)).get(1))
    lat2 = ee.Number(ee.List(imageBBOX.get(1)).get(1))
    lat3 = ee.Number(ee.List(imageBBOX.get(2)).get(1))
    lat4 = ee.Number(ee.List(imageBBOX.get(3)).get(1))
    lat5 = ee.Number(ee.List(imageBBOX.get(4)).get(1))

    # Make a smaller bounding box 
    return image.clip(ee.Geometry.Polygon( 
                              coords =  [ \
                                          ee.Geometry.LinearRing( \
                                            [ \
                                              [long1, lat1.add(clipDegrees)], \
                                              [long2, lat2.add(clipDegrees)], \
                                              [long3, lat3.subtract(clipDegrees)], \
                                              [long4, lat4.subtract(clipDegrees)], \
                                              [long5, lat5.add(clipDegrees)] \
                                            ]
                                          )
                                        ],
                                proj =  image.projection(),
                                geodesic =  False,
                                maxError = 0.1));


def numberOfPixels(img) :
    imgDescription = ee.Algorithms.Describe( img )
    height = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(0)
    width = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(1)
    return  ee.Number( width ).multiply( ee.Number( height ) )

def numberOfPixelsW(img) :
    imgDescription = ee.Algorithms.Describe( img )
    width = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(1)
    return  ee.Number( width )

def numberOfPixelsH(img) :
    imgDescription = ee.Algorithms.Describe( img )
    height = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(0)
    return  ee.Number(height )

In [145]:
# coded LUT for output MODIS MCD15
def getLUTout(imageMCD15MOD09):
    imageMCD15MOD09 = ee.Image(imageMCD15MOD09);

    return imageMCD15MOD09.select('Lai').min(99).multiply(1).toInt64() \
                        .add(imageMCD15MOD09.select('LaiStdDev').min(99).multiply(1e2).toInt64()) \
                        .add(imageMCD15MOD09.select('Fpar').min(99).multiply(1e4).toInt64()) \
                        .add(imageMCD15MOD09.select('FparStdDev').min(99).multiply(1e6).toInt64()) \
                        .rename('LUTout')

In [146]:
# coded LUT for input landsat image
def getLUTL0Xin(imageMCD15):
    imageMCD15 = ee.Image(imageMCD15);
    return imageMCD15.select('SR_B3').multiply(2.75e-05).add(0.02).max(0).min(0.999).multiply(1e3).floor().multiply(1).toInt64() \
                        .add(imageMCD15.select('SR_B4').multiply(2.75e-05).add(0.02).max(0).min(0.999).multiply(1000).floor().multiply(1e6).toInt64()) \
                        .add(imageMCD15.select('VIEW_ZENITH').multiply(0.01).multiply(0.5).multiply(10).floor().multiply(1e9).toInt64()) \
                        .add(ee.Image.constant(0).multiply(0.01).add(180).multiply(0.25).floor().multiply(1e11).toInt64()) \
                        .add(ee.Image.constant(90).subtract(ee.Image(imageMCD15.getNumber('SUN_ELEVATION'))).multiply(0.01).multiply(0.5).multiply(10).floor().multiply(1e14).toInt64()) \
                        .add(ee.Image(imageMCD15.getNumber('SUN_AZIMUTH')).multiply(0.01).add(180).multiply(0.25).floor().multiply  (1e16).toInt64()) \
                        .add(imageMCD15.select('LC_Type3').multiply(1e18).toInt64()) \
                        .rename('LUTL0Xin')







In [148]:
# sample worldwide 
# check each year and month
nsamSat = 0
nsamunSat = 0
for year in range(2013,2021):#

        
    # specify land cover projected into MOD15 projection and aggregated using median to outputScale    
    projectionMCD15=  ee.ImageCollection('MODIS/006/MCD15A3H').first().select('Lai').projection()
    projectionMCD12 =  ee.ImageCollection("MODIS/006/MCD12Q1").first().select('LC_Type3').projection()
    imageMCD12Q1 = ee.ImageCollection("MODIS/006/MCD12Q1").filterDate(ee.Date.fromYMD(year,1,1),ee.Date.fromYMD(year,12,31)).first() \
                     .select('LC_Type3') \
                     .reproject(crs= projectionMCD15,crsTransform= None,scale=projectionMCD12.nominalScale() ) \
                     .reduceResolution(reducer= ee.Reducer.median(),maxPixels= 1024,bestEffort=False) \
                     .reproject(crs= projectionMCD15.atScale(outputScale))      

    # sample monthly data
    for month in [6,7,5,8,4,9,3,10,2,11,1,12]:  
        
        #sample WRS path rows
        for path in range(6,233,deltaPath): 
            for row in range(22,248,deltaRow): 
                
                # sample deltaDay intervals of each month
                for daystart in range (1,30-deltaDay,deltaDay):
                    
                    # find Landsat images for paht , row and date interval between 0 and maxCloudCover
                    imageLC = ee.ImageCollection(collectionLandsat).filterMetadata('WRS_PATH','equals',path) \
                                                                          .filterMetadata('WRS_ROW','equals',row) \
                                                                          .filterDate(ee.Date.fromYMD(year,month,daystart),ee.Date.fromYMD(year,month,daystart).advance(deltaDay,'days')) \
                                                                          .filter(ee.Filter.lte('CLOUD_COVER_LAND',cloudCoverL0X))
                    # only proceed if there is at least one filtered landsat image (we will only use the first one found)
                    if (imageLC.size().getInfo() > 0) : 

                        # only sample the first scene we found in the interval of deltaDay's 
                        imageLC = imageLC.first()
 
                        
                        # get date range of imageLC and print it out
                        d1 = ee.Date(imageLC.get('system:time_start'))
                        d2 = ee.Date(imageLC.get('system:time_end'))     
                        print('year',year,'month',month,'path',path,'row',row)

                        # get  MOD15 product containing the aquistion of the  Landsat image
                        imageMCD15 = ee.ImageCollection('MODIS/006/MCD15A3H').map(lambda image: image.set('daterange',ee.DateRange(ee.Date(image.get('system:time_start')),ee.Date(image.get('system:time_end')))))\
                                                            .filter(ee.Filter.dateRangeContains('daterange', d1)) 

                        # only proceed if there is a MCD15 product containing the Landsat image aquisition
                        if (imageMCD15.size().getInfo() > 0)  :
                    
                            #randomly sample a sub-image to avoid memory issues, 
                            # add a validPixel we will use to determine aggregation and unmask since we will accept some missing data
                            imageGRID = imageLC.geometry().coveringGrid(projectionMCD15,subimageSize).randomColumn('randomIndex')                                               
                            imageLC = imageLC.addBands(getVZALC(imageLC).rename('VIEW_ZENITH')) \
                                             .clip(ee.Feature(imageGRID.sort('randomIndex').first()).geometry()) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<0).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<1).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<2).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<3).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<4).eq(0)) 
        
                            
                            # get the fraction of area with unmasked LC data 
                            validArea = ee.Number(ee.Image.pixelArea().mask(imageLC.select('QA_PIXEL').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=30,bestEffort=True).get('area')).getInfo()
                            totalArea = ee.Number(ee.Image.pixelArea().mask(imageLC.select('QA_PIXEL').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=30,bestEffort=True).get('area')).getInfo()
                            fracAreaLC = validArea / totalArea
                            #print('fracAreaLC:',fracAreaLC)  

                            # proceed if there is sufficient landsat area
                            if ( fracAreaLC > minfracAreaLC) :
                                                
                                # get MCD15 image from collection, clip to Landsat subimage
                                imageMCD15 = imageMCD15.first().clip(imageLC.geometry()) 
                                                
                                # update mask and add land cover and a saturation flag band
                                imageMCD15 = imageMCD15.updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<0).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<2).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<3).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<4).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<6).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<7).eq(0)) \
                                                .addBands( imageMCD12Q1 ).addBands(imageMCD15.select('FparLai_QC').rename(['satFlag']).bitwiseAnd(1<<5).eq(1))
                            
                                # get the fraction of area with unmasked MCD15 data 
                                validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),crs=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                                totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),crs=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                                fracAreaMCD15 = validArea / totalArea
                                #print('fracAreaMCD15:',fracAreaMCD15) 
                           
                                # sample if there is sufficient unmasked area
                                if ( fracAreaMCD15> minfracAreaMCD15 ) :
                                                
                                    
                                    # add a view zenith angle band and a validPixel band to the landsat image
                                    # then unmask and aggregate to output scale, we unmask first to allow for <100% validPixel s
                                    imageLCtoMCD = imageLC.addBands(imageLC.select('QA_PIXEL').gt(-1).rename('validPixel')) \
                                                         .unmask() \
                                                         .reproject(crs= projectionMCD15,scale=imageLC.projection().nominalScale() ) \
                                                         .reduceResolution(reducer= ee.Reducer.mean(),maxPixels=maxPixels,bestEffort=False) \
                                                         .reproject(crs= projectionMCD15 , scale= outputScale) 
                                    
                                    # mask bands not meeting fractional pixel threshold
                                    # we then have to divide by the fractional validPixel band to adjust for  zero masked values 
                                    validLCtoMCD = imageLCtoMCD.select('validPixel').gte(minfracValidLC);
                                    imageLCtoMCD = imageLCtoMCD.updateMask(validLCtoMCD) \
                                                     .divide(imageLCtoMCD.select('validPixel'))


                                    # add the aggregated LC image to the modis image and update the MCD image with the valid mask
                                    imageMCD15 = imageMCD15.addBands(imageLCtoMCD) \
                                                            .updateMask(validLCtoMCD) \
                                                            .set('SUN_AZIMUTH', imageLC.getNumber('SUN_AZIMUTH')) \
                                                             .set('SUN_ELEVATION', imageLC.getNumber('SUN_ELEVATION'))   

                                    # get the fraction of area with unmasked LC data 
                                    validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('QA_PIXEL').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageMCD15.geometry(),scale=outputScale,bestEffort=True).get('area')).getInfo()
                                    totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('QA_PIXEL').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageMCD15.geometry(),scale=outputScale,bestEffort=True).get('area')).getInfo()
                                    fracAreaLCtoMCD= validArea / totalArea
                                    #print('fracAreaLCtoMCD:',fracAreaLCtoMCD)  
                                    
                                    # check if we have enough valid cloud free aggregated matches
                                    if ( fracAreaLCtoMCD > minfracAreaLCtoMCD ) :
 

                                        imageMCD15 = imageMCD15.select('satFlag').addBands(getLUTout(imageMCD15)).addBands(getLUTL0Xin(imageMCD15))
                                        unsatSample = imageMCD15.updateMask(imageMCD15.select('satFlag').lt(ee.Image.constant(0.1))).sample(projection=projectionMCD15.atScale(outputScale), numPixels=maxSample, dropNulls=True)
                                        unsatSampleoutput = unsatSample.aggregate_array('LUTout').flatten().getInfo()
                                        unsatSampleLCinput = unsatSample.aggregate_array('LUTL0Xin').flatten().getInfo()
                                        if (len(unsatSampleoutput)>0) :
                                            
                                            unsatLUToutput = unsatLUToutput + unsatSampleoutput
                                            unsatLUTLCinput = unsatLUTLCinput + unsatSampleLCinput

                                            nsam = len(unsatSampleoutput)
                                            nsamunSat = nsamunSat + nsam
                                            print('cumulative unsat samples:',nsamunSat,'nsaminput unsat',len(unsatSampleLCinput))
                                            Year_ann_LX.append([year]*nsam)
                                            Month_ann_LX.append([month]*nsam)
                                            Daystart_ann_LX.append([daystart]*nsam)
                                            Path_ann_LX.append([path]*nsam)
                                            Row_ann_LX.append([row]*nsam)
   

                                        satSample = imageMCD15.updateMask(imageMCD15.select('satFlag').gt(ee.Image.constant(0.9))).sample(projection=projectionMCD15.atScale(outputScale), numPixels=maxSample, dropNulls=True)
                                        satSampleoutput = satSample.aggregate_array('LUTout').flatten().getInfo()
                                        satSampleLCinput = satSample.aggregate_array('LUTL0Xin').flatten().getInfo()
                                        if len(satSampleoutput )>0:
                                            
                                            satLUToutput = satLUToutput + satSampleoutput
                                            satLUTLCinput = satLUTLCinput + satSampleLCinput

                                            nsam = len(unsatSampleoutput)
                                            nsamSat = nsamSat + nsam
                                            print('cumulative sat samples :',nsamSat,'nsaminput sat ',len(satSampleLCinput))
                                            Year_ann_LXsat.append([year]*nsam)
                                            Month_ann_LXsat.append([month]*nsam)
                                            Daystart_ann_LXsat.append([daystart]*nsam)
                                            Path_ann_LXsat.append([path]*nsam)
                                            Row_ann_LXsat.append([row]*nsam)
                                            

        # write list after each month is done
        with open(, 'ab') as fp1:
            pickle.dump(unsatLUTLCinput, fp1)
        with open('/home/richard_fernandes/unsatLUToutputLC0X', 'ab') as fp2:
            pickle.dump(unsatLUToutput, fp2)
        with open('/home/richard_fernandes/satLUTinputLC0X', 'ab') as fp3:
            pickle.dump(satLUTLCinput, fp3)
        with open('/home/richard_fernandes/satLUToutputLC0X', 'ab') as fp4:
            pickle.dump(satLUToutput, fp4)
        print('done')

year 2013 month 6 path 6 row 22
year 2013 month 6 path 6 row 22
year 2013 month 6 path 6 row 23
year 2013 month 6 path 6 row 23


KeyboardInterrupt: 

In [ ]:
  # check the sampling worked
    with open('/home/richard_fernandes/unsatLUToutputLC0X', 'rb') as fp1:
    x = pickle.load(fp1)
    print(x)